In [1]:
import glob
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import pickle
import sys

# From arm
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.optim as optim
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from sklearn.linear_model import Lasso, LassoCV, LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm

from MouseArmTransformer import (
    criterion,
    data,
    helper,
    inference,
    model,
    training,
)
from mausspaun.data_processing.dlc import (
    DLC_TO_MUJOCO_MAPPING,
    MUJOCO_TO_DLC_MAPPING,
    align_data_with_rig_markers,
)
from MouseArmTransformer.gui import app
from mausspaun.visualization.plot_3D_video import plot_3d_video, plot_split_3d_video

%load_ext autoreload
%autoreload 2

In [ ]:
mouse_name = 'HoneyBee'
day = 77
attempt = 1
part = 0
base_path = '/data/mausspaun/'

In [ ]:
# Test inference
(X_3d_test, X_2d_c1_test, X_2d_c2_test, cam_positions, likelihood_c1, likelihood_c2) = helper.load_single_session()
cam_positions_dict = {key: X_2d_c1_test[:, i, :] for i, key in enumerate(helper.mausspaun_keys)}
inference_preds = inference.run_inference(cam_positions_dict)

In [ ]:
avg_error, errors = inference.evaluate_ground_truth(inference_preds, mouse_name, day, attempt, verbose=2)

---
# Test triangulated 3D without using the transformer

In [ ]:
# Original triangulated:
filepaths = app.generate_filepaths(base_path + 'videos/videos_dlc2/', mouse_name, day, attempt, part)
X_3d_train, X_2d_c1_train, X_2d_c2_train, cam_positions, likelihood_c1, likelihood_c2 = helper.get_training_data(
    filepaths[0][1], filepaths[1][1], likelihood_cutoff=0, convert_to_mujoco=False)
X_3d_train_dict = {key: X_3d_train[:, i, :] for i, key in enumerate(helper.mausspaun_keys)}
X_3d_train_mujoco, (dlc_c, dlc_s, T, mujoco_c, mujoco_s) = helper.dlc_to_mujoco(X_3d_train_dict.copy())

In [ ]:
avg_error, errors = inference.evaluate_ground_truth(X_3d_train_mujoco, mouse_name, day, attempt, verbose=2)

---

---
# Project back to DLC space, then calculate reprojection error

In [ ]:
inference_dlc = helper.mujoco_to_dlc(inference_preds.copy(), dlc_c, dlc_s, T, mujoco_c, mujoco_s)
inference_dlc_np = np.transpose(np.array([cp for key, cp in inference_dlc.items()]), axes=(1, 0, 2))
triangulated_3D_dlc = np.transpose(np.array([cp for key, cp in X_3d_train_dict.copy().items()]), axes=(1, 0, 2))

(all_cam1_transformer, all_cam2_transformer) = helper.project_from_3D_dlc(inference_dlc_np)
(all_cam1_triangulated, all_cam2_triangulated) = helper.project_from_3D_dlc(triangulated_3D_dlc)

errors_transformer_cam1 = helper.calculate_euclidean_error(X_2d_c1_train, all_cam1_transformer)
errors_transformer_cam2 = helper.calculate_euclidean_error(X_2d_c2_train, all_cam2_transformer)
errors_triangulated_cam1 = helper.calculate_euclidean_error(X_2d_c1_train, all_cam1_triangulated)
errors_triangulated_cam2 = helper.calculate_euclidean_error(X_2d_c2_train, all_cam2_triangulated)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 5), sharex=True, sharey=True)
axes = axes.flatten()

for idx, (errors, txt) in enumerate(
        zip([errors_transformer_cam1, errors_transformer_cam2, errors_triangulated_cam1, errors_triangulated_cam2],
            ['Transformer Cam1', 'Transformer Cam2', 'Triangulated Cam1', 'Triangulated Cam2'])):
    axes[idx].matshow(np.log(errors.T), aspect='auto')
    axes[idx].set_title(txt)
axes[0].set_ylabel('Joints')
axes[2].set_xlabel('Time')
fig.tight_layout()